# word2vec, fastTextで日本語学習済みモデルを使用する方法

## word2vec

In [1]:
# 7-2と一緒
import MeCab

m_t = MeCab.Tagger('-Ochasen -d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd')

def tokenizer_mecab(text):
    text = m_t.parse(text)  # これでスペースで単語が区切られる
    ret = text.strip().split()  # スペース部分で区切ったリストに変換
    return ret

# 前処理として正規化をする関数を定義
import re
def preprocessing_text(text):
    # 改行、半角スペース、全角スペースを削除
    text = re.sub('\r', '', text)
    text = re.sub('\n', '', text)
    text = re.sub('　', '', text)
    text = re.sub(' ', '', text)

    # 数字文字の一律「0」化
    text = re.sub(r'[0-9 ０-９]', '0', text)  # 数字

    return text


# 前処理とJanomeの単語分割を合わせた関数を定義する
def tokenizer_with_preprocessing(text):
    text = preprocessing_text(text)  # 前処理の正規化
    ret = tokenizer_mecab(text)  # Mecabの単語分割

    return ret

In [2]:
import torchtext

# tsvやcsvデータを読み込んだときに、読み込んだ内容に対して行う処理を定義します
# 文章とラベルの両方に用意します

max_length = 25
TEXT = torchtext.data.Field(sequential=True, tokenize=tokenizer_with_preprocessing,
                            use_vocab=True, lower=True, include_lengths=True, batch_first=True, fix_length=max_length)
LABEL = torchtext.data.Field(sequential=False, use_vocab=False)


# フォルダ「data」から各tsvファイルを読み込みます
train_ds, val_ds, test_ds = torchtext.data.TabularDataset.splits(
    path='./data/', train='text_train.tsv',
    validation='text_val.tsv', test='text_test.tsv', format='tsv',
    fields=[('Text', TEXT), ('Label', LABEL)])

In [3]:
from gensim.models import KeyedVectors
# 一度gensimライブラリで読み込んで、word2vecのformatで保存する
model = KeyedVectors.load_word2vec_format(
    './data/entity_vector/entity_vector.model.bin', binary=True)

# 保存（時間がかかります、10分弱）
model.wv.save_word2vec_format('./data/japanese_word2vec_vectors.vec')

ipykernel_launcher:7: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


In [4]:
from torchtext.vocab import Vectors

japanese_word2vec_vectors = Vectors(
    name='./data/japanese_word2vec_vectors.vec')

# 単語ベクトルの中身を確認します
print("1単語を表現する次元数：", japanese_word2vec_vectors.dim)
print("単語数：", len(japanese_word2vec_vectors.itos))

1単語を表現する次元数： 200
単語数： 1015474


In [5]:
TEXT.build_vocab(train_ds, vectors=japanese_word2vec_vectors, min_freq=1)

# ボキャブラリーのベクトルを確認
print(TEXT.vocab.vectors.shape)  # 49個の単語が200次元のベクトルで表現
TEXT.vocab.vectors

torch.Size([123, 200])


tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [6]:
TEXT.vocab.stoi

defaultdict(<bound method Vocab._default_unk_index of <torchtext.vocab.Vocab object at 0x7f8234940d68>>,
            {'<unk>': 0,
             '<pad>': 1,
             '名詞-一般': 2,
             '。': 3,
             'と': 4,
             '助詞-格助詞-一般': 5,
             '、': 6,
             '助動詞': 7,
             'の': 8,
             '基本形': 9,
             '文章': 10,
             'が': 11,
             'を': 12,
             '動詞-自立': 13,
             '名詞-サ変接続': 14,
             '連用形': 15,
             'いる': 16,
             'する': 17,
             'ます': 18,
             'ト': 19,
             '助詞-並立助詞': 20,
             'eos': 21,
             'か': 22,
             'だ': 23,
             'て': 24,
             'な': 25,
             'サ変・スル': 26,
             'ナ': 27,
             'ノ': 28,
             'ブンショウ': 29,
             '体言接続': 30,
             '分類': 31,
             '本章': 32,
             '特殊・ダ': 33,
             '記号-句点': 34,
             '評価': 35,
             '0': 36,
             'し': 37,


In [7]:
'''
    '女性': 72,
    '女王': 73,
    '好き': 74,
    '姫': 75,
    '構築': 76,
    '機械学習': 77,
    '王': 78,
    '王子': 79,
    '男性': 80,
'''

# 姫 - 女性 + 男性 のベクトルがどれと似ているのか確認
import torch.nn.functional as F

# 姫 - 女性 + 男性
tensor_calc = TEXT.vocab.vectors[75] - \
    TEXT.vocab.vectors[72] + TEXT.vocab.vectors[80]

# コサイン類似度を計算
# dim=0 は0次元目で計算してくださいという指定
print("女王", F.cosine_similarity(tensor_calc, TEXT.vocab.vectors[73], dim=0))
print("王", F.cosine_similarity(tensor_calc, TEXT.vocab.vectors[78], dim=0))
print("王子", F.cosine_similarity(tensor_calc, TEXT.vocab.vectors[79], dim=0))
print("機械学習", F.cosine_similarity(tensor_calc, TEXT.vocab.vectors[77], dim=0))

女王 tensor(0.3840)
王 tensor(0.3669)
王子 tensor(0.5489)
機械学習 tensor(-0.1404)


## fastText

In [8]:
from torchtext.vocab import Vectors

japanese_fasttext_vectors = Vectors(name='./data/vector_neologd/model.vec')
                              
# 単語ベクトルの中身を確認します
print("1単語を表現する次元数：", japanese_fasttext_vectors.dim)
print("単語数：", len(japanese_fasttext_vectors.itos))

1単語を表現する次元数： 300
単語数： 351122


In [9]:
# ベクトル化したバージョンのボキャブラリーを作成
TEXT.build_vocab(train_ds, vectors=japanese_fasttext_vectors, min_freq=1)

# ボキャブラリーのベクトルを確認
print(TEXT.vocab.vectors.shape)  # 52個の単語が300次元のベクトルで表現
TEXT.vocab.vectors

# ボキャブラリーの単語の順番を確認
TEXT.vocab.stoi

torch.Size([123, 300])


defaultdict(<bound method Vocab._default_unk_index of <torchtext.vocab.Vocab object at 0x7f8227ebe898>>,
            {'<unk>': 0,
             '<pad>': 1,
             '名詞-一般': 2,
             '。': 3,
             'と': 4,
             '助詞-格助詞-一般': 5,
             '、': 6,
             '助動詞': 7,
             'の': 8,
             '基本形': 9,
             '文章': 10,
             'が': 11,
             'を': 12,
             '動詞-自立': 13,
             '名詞-サ変接続': 14,
             '連用形': 15,
             'いる': 16,
             'する': 17,
             'ます': 18,
             'ト': 19,
             '助詞-並立助詞': 20,
             'eos': 21,
             'か': 22,
             'だ': 23,
             'て': 24,
             'な': 25,
             'サ変・スル': 26,
             'ナ': 27,
             'ノ': 28,
             'ブンショウ': 29,
             '体言接続': 30,
             '分類': 31,
             '本章': 32,
             '特殊・ダ': 33,
             '記号-句点': 34,
             '評価': 35,
             '0': 36,
             'し': 37,


In [10]:
# 姫 - 女性 + 男性 のベクトルがどれと似ているのか確認
import torch.nn.functional as F

# 姫 - 女性 + 男性
tensor_calc = TEXT.vocab.vectors[75] - \
    TEXT.vocab.vectors[72] + TEXT.vocab.vectors[80]

# コサイン類似度を計算
# dim=0 は0次元目で計算してくださいという指定
print("女王", F.cosine_similarity(tensor_calc, TEXT.vocab.vectors[73], dim=0))
print("王", F.cosine_similarity(tensor_calc, TEXT.vocab.vectors[78], dim=0))
print("王子", F.cosine_similarity(tensor_calc, TEXT.vocab.vectors[79], dim=0))
print("機械学習", F.cosine_similarity(tensor_calc, TEXT.vocab.vectors[77], dim=0))

女王 tensor(0.3650)
王 tensor(0.3461)
王子 tensor(0.5531)
機械学習 tensor(0.0952)
